<a href="https://colab.research.google.com/github/ABHAY1937/NLP/blob/main/FackNewsClassification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [102]:
import csv
import sys
import os

# Set the field size limit to a larger value
csv.field_size_limit(sys.maxsize)

data = []
file_path = 'fake news dataset.csv'
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='latin1') as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)

    df = pd.DataFrame(data[1:], columns=data[0])
else:
    print(f"File not found: {file_path}")


File not found: fake news dataset.csv


In [103]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
8159,8159,Report: â72 Terrorists Came From Countries C...,Michael Patrick Leahy,âA review of information compiled by a Senat...,0
8160,8160,Trumpâs $1.6 Billion Budget for Border Walls...,Bob Price,President Donald Trumpâs 2018 budget asks Co...,0
8161,8161,World War 3 is on the Horizon,Pakalert,"October 31, 2016 at 5:53 am \nI grew up in a r...",1
8162,8162,FBI To Scour 650k Emails Linked To Private Ser...,Edmondo Burr,RT.com reports: \nThe search warrant allows th...,1


In [104]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [105]:
df.shape

(8164, 5)

In [106]:
#finding the null values
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [107]:
#droping nan values
df = df.dropna()

In [108]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [109]:
#Get the Independent Features
x = df.drop('label',axis=1)

In [110]:
#get the Dependent feature
y = df['label']

In [111]:
x.shape

(8164, 4)

In [112]:
y.shape

(8164,)

In [113]:
import tensorflow as tf

In [114]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [115]:
#Velocity size
vox_size=5000

OneHot representation

In [116]:
message = x.copy()

In [117]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [118]:
message


,id,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
8159,8159,Report: â72 Terrorists Came From Countries C...,Michael Patrick Leahy,âA review of information compiled by a Senat...
8160,8160,Trumpâs $1.6 Billion Budget for Border Walls...,Bob Price,President Donald Trumpâs 2018 budget asks Co...
8161,8161,World War 3 is on the Horizon,Pakalert,"October 31, 2016 at 5:53 am \nI grew up in a r..."
8162,8162,FBI To Scour 650k Emails Linked To Private Ser...,Edmondo Burr,RT.com reports: \nThe search warrant allows th...


In [119]:
#message.reset_index(inplace=True)

In [120]:
import nltk
import re
from nltk.corpus import stopwords

In [121]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [122]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [123]:
corpus


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [124]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [125]:
vocabulary = set(corpus)
voc_size = len(vocabulary)
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[982, 7284, 6877, 6112, 5690, 7538, 4402, 2807, 2915, 1059],
 [4392, 3388, 6454, 1661, 6511, 204, 426],
 [5274, 3003, 981, 265],
 [4442, 7227, 5149, 4627, 775, 7058],
 [1520, 6511, 1307, 2028, 6247, 5331, 6511, 4202, 7160, 1915],
 [2304,
  6637,
  3247,
  4843,
  6283,
  2784,
  4683,
  3110,
  1822,
  6692,
  104,
  7037,
  3729,
  919,
  426],
 [2062, 2062, 2082, 5052, 1113, 523, 1159, 6499, 4589, 1096, 7527, 1872],
 [815, 1427, 3842, 1148, 4289, 7679, 2746, 276, 4562, 6199, 4032],
 [2480,
  3670,
  3532,
  4876,
  2784,
  6653,
  3692,
  4619,
  1512,
  1147,
  4562,
  6199,
  4032],
 [232, 2864, 785, 262, 4448, 2576, 2784, 5844, 4562, 6199, 4032],
 [3977, 6691, 7475, 5408, 1730, 6156, 1366, 4196, 2784, 5106],
 [2486, 5852, 4470, 7581, 650, 1789, 3252, 7103],
 [4286, 6878, 7581, 7747, 167, 7015, 5165, 6818, 765, 6431, 1972],
 [4627, 6097, 5690, 6156, 2784, 4448],
 [5537, 7331, 7435, 4616, 6341, 439, 2538, 4365, 4839],
 [1493, 7585, 4204, 2397, 1804, 4600, 7139, 4562, 6199, 4032],
 

In [126]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [127]:
onehot_repr[1]

[4392, 3388, 6454, 1661, 6511, 204, 426]

Embedding Representation

In [128]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 982 7284 6877 ...    0    0    0]
 [4392 3388 6454 ...    0    0    0]
 [5274 3003  981 ...    0    0    0]
 ...
 [4119 6357 3481 ...    0    0    0]
 [3097 5161 6430 ...    0    0    0]
 [4223  526 6188 ...    0    0    0]]


In [129]:
embedded_docs[1]

array([4392, 3388, 6454, 1661, 6511,  204,  426,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [130]:
embedded_docs[0]

array([ 982, 7284, 6877, 6112, 5690, 7538, 4402, 2807, 2915, 1059,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [131]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            310200    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 366,701
Trainable params: 366,701
Non-trainable params: 0
_________________________________________________________________
None


In [132]:
len(embedded_docs),y.shape

(8164, (8164,))

In [133]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [134]:
x_final.shape,y_final.shape

((8164, 20), (8164,))

In [135]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

Model Training

In [136]:
# Convert target variable to float
y_train = y_train.astype(float)
y_test = y_test.astype(float)

# Finally Training
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
86/86 [==============================] - 7s 50ms/step - loss: 0.4179 - accuracy: 0.7914 - val_loss: 0.2310 - val_accuracy: 0.9106
Epoch 2/10
86/86 [==============================] - 4s 45ms/step - loss: 0.1321 - accuracy: 0.9515 - val_loss: 0.2247 - val_accuracy: 0.9165
Epoch 3/10
86/86 [==============================] - 5s 58ms/step - loss: 0.0586 - accuracy: 0.9812 - val_loss: 0.2791 - val_accuracy: 0.9117
Epoch 4/10
86/86 [==============================] - 4s 44ms/step - loss: 0.0331 - accuracy: 0.9916 - val_loss: 0.4034 - val_accuracy: 0.9113
Epoch 5/10
86/86 [==============================] - 4s 45ms/step - loss: 0.0208 - accuracy: 0.9936 - val_loss: 0.3499 - val_accuracy: 0.8894
Epoch 6/10
86/86 [==============================] - 9s 102ms/step - loss: 0.0176 - accuracy: 0.9951 - val_loss: 0.5186 - val_accuracy: 0.9050
Epoch 7/10
86/86 [==============================] - 4s 43ms/step - loss: 0.0085 - accuracy: 0.9982 - val_loss: 0.4898 - val_accuracy: 0.9165
Epoch 8/10
8

Adding Dropout


In [137]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Performance Metrics And Accuracy

In [144]:
y_pred=model.predict(x_test)

85/85 [==============================] - 1s 10ms/step


In [145]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [146]:
from sklearn.metrics import confusion_matrix

In [147]:
confusion_matrix(y_test,y_pred)

array([[1412,    0],
       [1283,    0]])

In [148]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5239332096474953

In [149]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.52      1.00      0.69      1412
         1.0       0.00      0.00      0.00      1283

    accuracy                           0.52      2695
   macro avg       0.26      0.50      0.34      2695
weighted avg       0.27      0.52      0.36      2695



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
